In [1]:
import cmath
import pandas as pd
import numpy as np

class data:
  def __init__(self, bus, line):
    self.bus = bus
    self.line = line

case = 1
if case == 1:
    data.bus = pd.DataFrame({'bus': ['bus1', 'bus2', 'bus3'],
                         'type': ['slack', 'PV', 'PQ'],
                         'p_pu': [np.nan, 150/100, 250/100],
                         'q_pu': [np.nan, np.nan, 100/100],
                         'v_pu': [1.0, 1.01, np.nan]})
    data.line = pd.DataFrame({'from_bus': [0, 0, 1],
                              'to_bus': [1, 2, 2],
                              'r_pu': [0, 0.01, 0.0125],
                              'x_pu': [0.04, 0.03, 0.025]})
elif case == 2:
    data.bus = pd.DataFrame({'bus': ['bus1', 'bus2', 'bus3'],
                         'type': ['slack', 'PQ', 'PV'],
                         'p_pu': [np.nan, 400/100, 200/100],
                         'q_pu': [np.nan, 250/100, np.nan],
                         'v_pu': [1.05, np.nan, 1.04]})
    data.line = pd.DataFrame({'from_bus': [0, 0, 1],
                              'to_bus': [1, 2, 2],
                              'r_pu': [0.02, 0.01, 0.0125],
                              'x_pu': [0.04, 0.03, 0.025]})


In [2]:
Y = np.zeros([data.bus.shape[0], data.bus.shape[0]]).astype(complex)
for irow in range(data.bus.shape[0]):
    r_pu = []
    x_pu = []
    if data.line['r_pu'][(data.line['from_bus']==irow)].shape[0] != 0:
        r_pu = data.line['r_pu'][(data.line['from_bus']==irow)].to_list()
        x_pu = data.line['x_pu'][(data.line['from_bus']==irow)].to_list()
    if data.line['r_pu'][(data.line['to_bus']==irow)].shape[0] != 0:
        r_pu = np.append(r_pu, data.line['r_pu'][(data.line['to_bus']==irow)].to_list())
        x_pu = np.append(x_pu, data.line['x_pu'][(data.line['to_bus']==irow)].to_list())
    Y[irow][irow] = np.sum([1/complex(r_pu[idx], x_pu[idx]) for idx in range(len(r_pu))])
    for icol in range(data.bus.shape[0]):
        if irow != icol:
            try:
                r_pu = data.line['r_pu'][(data.line['from_bus']==irow) & (data.line['to_bus']==icol)].item()
                x_pu = data.line['x_pu'][(data.line['from_bus']==irow) & (data.line['to_bus']==icol)].item()
            except:
                r_pu = data.line['r_pu'][(data.line['from_bus']==icol) & (data.line['to_bus']==irow)].item()
                x_pu = data.line['x_pu'][(data.line['from_bus']==icol) & (data.line['to_bus']==irow)].item()
            z_pu = 1/complex(r_pu, x_pu)
            Y[irow][icol] = -z_pu
Y

array([[ 10.-55.j,  -0.+25.j, -10.+30.j],
       [ -0.+25.j,  16.-57.j, -16.+32.j],
       [-10.+30.j, -16.+32.j,  26.-62.j]])

In [3]:
data.bus

,bus,type,p_pu,q_pu,v_pu
0,bus1,slack,NaN,NaN,1.00
1,bus2,PV,1.5,NaN,1.01
2,bus3,PQ,2.5,1.0,NaN


In [4]:
data.line

,from_bus,to_bus,r_pu,x_pu
0,0,1,0.0000,0.040
1,0,2,0.0100,0.030
2,1,2,0.0125,0.025


In [5]:
round_number = 4
iter_number = 5

V = np.zeros([1, data.bus.shape[0]]).astype('complex')
Q = np.zeros([1, data.bus.shape[0]])
P = np.zeros([1, data.bus.shape[0]])
delta = np.zeros([1, data.bus.shape[0]])
for i in range(V.shape[1]):
    if np.isnan(data.bus['v_pu'][i]):
        V[0,i] = complex(1, 0)
    else:
        V[0,i] = complex(data.bus['v_pu'][i], 0)
        
for i in range(Q.shape[1]):
    if np.isnan(data.bus['q_pu'][i]):
        Q[0,i] = np.nan
    else:
        Q[0,i] = data.bus['q_pu'][i]
        P[0,i] = data.bus['p_pu'][i]
        
for i in range(P.shape[1]):
    if np.isnan(data.bus['p_pu'][i]):
        P[0,i] = np.nan
    else:
        P[0,i] = data.bus['p_pu'][i]
        
for count in range(iter_number):
    Q = np.append(Q, np.zeros([1, data.bus.shape[0]]), axis=0)
    V = np.append(V, np.zeros([1, data.bus.shape[0]]), axis=0)
    for ical in range(data.bus.shape[0]):
        if data.bus['type'][ical] == 'PQ':
            Q[count+1, ical] = Q[count, ical]
            summation = 0
            for ioth in range(data.bus.shape[0]):
                if ical != ioth:
                    if V[count+1, ioth] > 0:
                        summation += (-Y[ical,ioth]*V[count+1, ioth])
                    else:
                        summation += (-Y[ical,ioth]*V[count, ioth])
            V[count+1, ical] = (1/Y[ical, ical])*((-complex(P[0, ical], Q[count+1, ical]).conjugate()/V[count, ical].conjugate())+summation)
            V[count+1, ical] = np.round(V[count+1, ical], round_number)

        elif data.bus['type'][ical] == 'PV':
            #Calculate Q
            summation = 0
            for ioth in range(data.bus.shape[0]):
                if V[count+1, ioth] > 0:
                    summation += (Y[ical,ioth]*V[count+1, ioth])
                else:
                    summation += (Y[ical,ioth]*V[count, ioth])
                
            Q[count+1, ical] = -((V[count, ical].conjugate()*summation).imag)
            Q[count+1, ical] = np.round(Q[count+1, ical],  round_number)

            #Calculate V
            summation = 0
            for ioth in range(data.bus.shape[0]):
                if ical != ioth:
                    if V[count+1, ioth] > 0:
                        summation += (-Y[ical,ioth]*V[count+1, ioth])
                    else:
                        summation += (-Y[ical,ioth]*V[count, ioth])
            Vc = (1/Y[ical, ical])*((complex(P[0, ical], Q[count+1, ical]).conjugate()/V[count, ical].conjugate())+summation)
            Vc = np.round(Vc, round_number)
            fc = Vc.imag
            ec = np.sqrt(abs(V[0,ical])**2-fc**2)
            V[count+1, ical] = complex(ec,fc)
            V[count+1, ical] = np.round(V[count+1, ical],  round_number)

        elif data.bus['type'][ical] == 'slack':
            V[count+1, ical] = V[count, ical]

display(V)

array([[1.    +0.j    , 1.01  +0.j    , 1.    +0.j    ],
       [1.    +0.j    , 1.0098+0.0216j, 0.9763-0.0167j],
       [1.    +0.j    , 1.01  +0.0085j, 0.9757-0.0239j],
       [1.    +0.j    , 1.01  +0.0035j, 0.9757-0.0263j],
       [1.    +0.j    , 1.01  +0.0018j, 0.9757-0.0271j],
       [1.    +0.j    , 1.01  +0.0013j, 0.9757-0.0274j]])

In [6]:
Q

array([[   nan,    nan, 1.    ],
       [0.    , 0.5757, 1.    ],
       [0.    , 0.7608, 1.    ],
       [0.    , 0.8528, 1.    ],
       [0.    , 0.8851, 1.    ],
       [0.    , 0.8968, 1.    ]])